In [1]:
import matplotlib.pyplot as plt
import scipy.stats as stats
from pyddm import Model
from pyddm import Sample
import pyddm as ddm
import numpy as np
import pandas as pd
import random
import os
import copy

## Fit M1: basic model

In [2]:
from pyddm import InitialCondition,Fittable, Fitted
from pyddm.models import Drift, Bound, LossRobustBIC, NoiseConstant, BoundConstant, OverlayChain, OverlayNonDecision, OverlayPoissonMixture
from pyddm.functions import fit_adjust_model

class ICPointSideBiasRatio(InitialCondition):
    name = "A side-biased starting point expressed as a proportion of the distance between the bounds."
    required_parameters = ["x0"]
    required_conditions = ["present"]
    def get_IC(self, x, dx, conditions):
        x0 = self.x0/2 + .5 #rescale to between 0 and 1
        # Bias > .5 for left side correct, bias < .5 for right side correct.
        # On original scale, positive bias for left, negative for right
        if not conditions['present']==1:
            x0 = 1-x0
        shift_i = int((len(x)-1)*x0)
        assert shift_i >= 0 and shift_i < len(x), "Invalid initial conditions"
        pdf = np.zeros(len(x))
        pdf[shift_i] = 1. # Initial condition at x=x0*2*B.
        return pdf
    
class DriftPresent(ddm.models.Drift):
    name = "Drift depends on target presence"
    required_parameters = ["driftpresent","driftabsent"] # <-- Parameters we want to include in the model
    required_conditions = ["present"] # <-- Task parameters ("conditions"). Should be the same name as in the sample.

    # We must always define the get_drift function, which is used to compute the instantaneous value of drift.
    def get_drift(self, conditions, **kwargs):
        # so that driftabsent values are normally negative
        return self.driftpresent * conditions['present'] - self.driftabsent * (1-conditions['present'])




In [6]:
m1_spec = Model(name='Basic model drift varies as a function of target presence',
                 drift=DriftPresent(driftabsent=Fittable(minval=-10, maxval=0),
                                     driftpresent=Fittable(minval=0, maxval=10)),
                 noise=NoiseConstant(noise=1),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundConstant(B=Fittable(minval=0.1, maxval=2.5)),
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m1(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m1_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftabsent': [float(fit_model.parameters()['drift']['driftabsent'])],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'noise': [float(fit_model.parameters()['noise']['noise'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondec': [float(fit_model.parameters()['overlay']['nondectime'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m1/data_"+data_label):
                       os.makedirs("model_fits/model_m1/data_"+data_label)
                       
    with open("model_fits/model_m1/data_"+data_label+"/subj"+str(subj_id)+".txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)

    
# loaded = pd.read_csv("data/E2.csv") 
# loaded['present'] = loaded['present'].replace(-1,0)
# summary_list = []
# for name, group in loaded.groupby('subj_id'):
#     print(name)
#     a= fit_m1(group.reset_index(),'E2')
#     summary_list.append(a)
# summary_df = pd.concat(summary_list)
# summary_df.to_csv("model_fits/model_m1/data_E2/all_subjects.csv")

## Fit M2: boundaries can collapse

In [7]:
from pyddm import InitialCondition,Fittable, Fitted
from pyddm.models import Drift, Bound, LossRobustBIC, NoiseConstant, OverlayChain, OverlayNonDecision, OverlayPoissonMixture, Noise
from pyddm.functions import fit_adjust_model

class BoundCollapse(Bound):
    name = "Boundary for the collapsing boundary model"
    required_parameters = ["B", "uppercollapse", "lowercollapse"]
    required_conditions = []
    def get_bound(self, t, conditions, **kwargs):
        return max(0.01, self.B + t*(self.uppercollapse-self.lowercollapse)/2)
    
class DriftPresentCollapse(ddm.models.Drift):
    name = "Drift depends on target presence and collapses"
    required_parameters = ["driftpresent","uppercollapse", "lowercollapse"] # <-- Parameters we want to include in the model
    required_conditions = ["present"] # <-- Task parameters ("conditions"). Should be the same name as in the sample.

    # We must always define the get_drift function, which is used to compute the instantaneous value of drift.
    def get_drift(self, t, conditions, **kwargs):
        # so that driftabsent values are normally negative
        return conditions['present'] * (self.driftpresent - (self.uppercollapse+self.lowercollapse)/2) + \
    (1-conditions['present']) *(0+ (self.uppercollapse+self.lowercollapse)/2)





In [8]:
uppercollapse = Fittable(minval=-5, maxval=0)
lowercollapse = Fittable(minval=0, maxval=5)

m2_spec = Model(name='Absence drift is zero boundaries can collapse',
                 drift=DriftPresentCollapse(driftpresent=Fittable(minval=0, maxval=10),
                                   uppercollapse=uppercollapse,
                                   lowercollapse=lowercollapse),
                 noise=NoiseConstant(noise=1),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundCollapse(B=Fittable(minval=0.1, maxval=2.5),
                                    uppercollapse=uppercollapse,
                                    lowercollapse=lowercollapse),
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m2(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m2_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'uppercollapse': [float(fit_model.parameters()['drift']['uppercollapse'])],
        'lowercollapse': [float(fit_model.parameters()['drift']['lowercollapse'])],
        'noise': [float(fit_model.parameters()['noise']['noise'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondec': [float(fit_model.parameters()['overlay']['nondectime'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m2/data_"+data_label):
                       os.makedirs("model_fits/model_m2/data_"+data_label)
                       
    with open("model_fits/model_m2/data_"+data_label+"/subj"+str(subj_id)+".txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)
        

# loaded = pd.read_csv("data/E2.csv") 
# loaded['present'] = loaded['present'].replace(-1,0)
# summary_list = []
# for name, group in loaded.groupby('subj_id'):
#     print(name)
#     a= fit_m2(group.reset_index(),'E2')
#     summary_list.append(a)
# summary_df = pd.concat(summary_list)
# summary_df.to_csv("model_fits/model_m2/data_E2/all_subjects.csv")

## Fit M3: Lower boundary collapses rapidly


In [10]:
class BoundNonlinearCollapse(Bound):
    name = "Lower boundary collapses nonlinearly"
    required_parameters = ["B", "lowercollapse", "collapsetime"]
    required_conditions = []
    def get_bound(self, t, conditions, **kwargs):
        if t<self.collapsetime:
            return(self.B)
        else:
            return max(0.01, self.B - (t-self.collapsetime)*self.lowercollapse/2)
    
class DriftPresentNonlinearCollapse(ddm.models.Drift):
    name = "Drift depends on target presence and collapses nonlinearly"
    required_parameters = ["driftpresent","lowercollapse", "collapsetime"] # <-- Parameters we want to include in the model
    required_conditions = ["present"] # <-- Task parameters ("conditions"). Should be the same name as in the sample.

    # We must always define the get_drift function, which is used to compute the instantaneous value of drift.
    def get_drift(self, t, conditions, **kwargs):
        if t<self.collapsetime:
            return conditions['present'] * (self.driftpresent)
        else:
            return conditions['present'] * (self.driftpresent-self.lowercollapse/2) + \
        (1-conditions['present']) * (0+ self.lowercollapse/2)


In [11]:
lowercollapse = Fittable(minval=0, maxval=10)
collapsetime = Fittable(minval=0, maxval=5)

m3_spec = Model(name='Absence drift is zero lower boundary can collapse nonlinearly',
                 drift=DriftPresentNonlinearCollapse(driftpresent=Fittable(minval=0, maxval=10),
                                   lowercollapse=lowercollapse,
                                   collapsetime=collapsetime),
                 noise=NoiseConstant(noise=1),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundNonlinearCollapse(B=Fittable(minval=0.1, maxval=2.5),
                                    lowercollapse=lowercollapse,
                                    collapsetime=collapsetime),
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m3(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m3_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'collapsetime': [float(fit_model.parameters()['drift']['collapsetime'])],
        'lowercollapse': [float(fit_model.parameters()['drift']['lowercollapse'])],
        'noise': [float(fit_model.parameters()['noise']['noise'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondec': [float(fit_model.parameters()['overlay']['nondectime'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m3/data_"+data_label):
                       os.makedirs("model_fits/model_m3/data_"+data_label)
                       
    with open("model_fits/model_m3/data_"+data_label+"/subj"+str(subj_id)+".txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)
        

# loaded = pd.read_csv("data/E2.csv") 
# loaded['present'] = loaded['present'].replace(-1,0)
# summary_list = []
# for name, group in loaded.groupby('subj_id'):
#     print(name)
#     a= fit_m3(group.reset_index(),'E2')
#     summary_list.append(a)
# summary_df = pd.concat(summary_list)
# summary_df.to_csv("model_fits/model_m3/data_E2/all_subjects.csv")

## Fit M4: unequal variance

In [12]:
class NoiseAsymmetric(Noise):
    name = "Noise is higher when a target is present"
    required_parameters = ["noiseabsent"]
    required_conditions = ['present']
    def get_noise(self, t, conditions, **kwargs):
        if conditions['present']==1:
            return 1
        else:
            return self.noiseabsent

In [13]:
m4_spec = Model(name='Basic model drift varies as a function of target presence',
                 drift=DriftPresent(driftabsent=Fittable(minval=-10, maxval=0),
                                     driftpresent=Fittable(minval=0, maxval=10)),
                 noise=NoiseAsymmetric(noiseabsent=Fittable(minval=0.5,maxval=2)),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundConstant(B=Fittable(minval=0.1, maxval=2.5)),
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m4(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m4_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftabsent': [float(fit_model.parameters()['drift']['driftabsent'])],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'noiseabsent': [float(fit_model.parameters()['noise']['noiseabsent'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondec': [float(fit_model.parameters()['overlay']['nondectime'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m4/data_"+data_label):
                       os.makedirs("model_fits/model_m4/data_"+data_label)
                       
    with open("model_fits/model_m4/data_"+data_label+"/subj"+str(subj_id)+".txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)

# loaded = pd.read_csv("data/E2.csv") 
# loaded['present'] = loaded['present'].replace(-1,0)
# summary_list = []
# for name, group in loaded.groupby('subj_id'):
#     print(name)
#     a= fit_m4(group.reset_index(),'E2')
#     summary_list.append(a)
# summary_df = pd.concat(summary_list)
# summary_df.to_csv("model_fits/model_m4/data_E2/all_subjects.csv")

## Fit M5: unequal variance + collapsing boundaries

In [14]:
m5_spec = Model(name='Absence drift is zero boundaries can collapse',
                 drift=DriftPresentCollapse(driftpresent=Fittable(minval=0, maxval=10),
                                   uppercollapse=uppercollapse,
                                   lowercollapse=lowercollapse),
                 noise=NoiseAsymmetric(noiseabsent=Fittable(minval=0.5,maxval=2)),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundCollapse(B=Fittable(minval=0.1, maxval=2.5),
                                    uppercollapse=uppercollapse,
                                    lowercollapse=lowercollapse),
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m5(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m5_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'uppercollapse': [float(fit_model.parameters()['drift']['uppercollapse'])],
        'lowercollapse': [float(fit_model.parameters()['drift']['lowercollapse'])],
        'noiseabsent': [float(fit_model.parameters()['noise']['noiseabsent'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondec': [float(fit_model.parameters()['overlay']['nondectime'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m5/data_"+data_label):
                       os.makedirs("model_fits/model_m5/data_"+data_label)
                       
    with open("model_fits/model_m5/data_"+data_label+"/subj"+str(subj_id)+".txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)
        

# loaded = pd.read_csv("data/E2.csv") 
# loaded['present'] = loaded['present'].replace(-1,0)
# summary_list = []
# for name, group in loaded.groupby('subj_id'):
#     print(name)
#     a= fit_m5(group.reset_index(),'E2')
#     summary_list.append(a)
# summary_df = pd.concat(summary_list)
# summary_df.to_csv("model_fits/model_m5/data_E2/all_subjects.csv")

## Fit M6: unequal variance + nonlinearly collapsing boundary

In [15]:
m6_spec = Model(name='Absence drift is zero lower boundary can collapse nonlinearly',
                 drift=DriftPresentNonlinearCollapse(driftpresent=Fittable(minval=0, maxval=10),
                                   lowercollapse=lowercollapse,
                                   collapsetime=collapsetime),
                 noise=NoiseAsymmetric(noiseabsent=Fittable(minval=0,maxval=3)),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundNonlinearCollapse(B=Fittable(minval=0.1, maxval=2.5),
                                    lowercollapse=lowercollapse,
                                    collapsetime=collapsetime),
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m6(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m6_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'collapsetime': [float(fit_model.parameters()['drift']['collapsetime'])],
        'lowercollapse': [float(fit_model.parameters()['drift']['lowercollapse'])],
        'noiseabsent': [float(fit_model.parameters()['noise']['noiseabsent'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondec': [float(fit_model.parameters()['overlay']['nondectime'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m6/data_"+data_label):
                       os.makedirs("model_fits/model_m6/data_"+data_label)
                       
    with open("model_fits/model_m6/data_"+data_label+"/subj"+str(subj_id)+".txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)
        

# loaded = pd.read_csv("data/E2.csv") 
# loaded['present'] = loaded['present'].replace(-1,0)
# summary_list = []
# for name, group in loaded.groupby('subj_id'):
#     print(name)
#     a= fit_m6(group.reset_index(),'E2')
#     summary_list.append(a)
# summary_df = pd.concat(summary_list)
# summary_df.to_csv("model_fits/model_m6/data_E2/all_subjects.csv")

## Fit M7: basic model with changing t0

In [34]:
from pyddm import OverlayNonDecision, Solution
class OverlayNonDecisionPresent(OverlayNonDecision):
    name = "Separate non-decision time for target presence or absence"
    required_parameters = ["nondectimepresent", "nondectimeabsent"]
    required_conditions = ['present']
    def get_nondecision_time(self, conditions):
        return self.nondectimepresent if conditions['present'] == 1 else self.nondectimeabsent

In [35]:
m7_spec = Model(name='Basic model drift varies as a function of target presence',
                 drift=DriftPresent(driftabsent=Fittable(minval=-10, maxval=0),
                                     driftpresent=Fittable(minval=0, maxval=10)),
                 noise=NoiseConstant(noise=1),
                 IC=ICPointSideBiasRatio(x0=Fittable(minval=-1, maxval=1)),
                 bound=BoundConstant(B=Fittable(minval=0.1, maxval=2.5)),
                 overlay=OverlayChain(overlays=[OverlayNonDecisionPresent(nondectimepresent=Fittable(minval=0, maxval=1),
                                                                         nondectimeabsent = Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.01, dt=.1, T_dur=5)

def fit_m7(df,data_label):
    
    df = df.reset_index();
    subj_id = df.subj_id[0]
    
    sample = Sample.from_pandas_dataframe(df, rt_column_name="rt", correct_column_name="correct")
    fit_model = fit_adjust_model(sample=sample, model=m7_spec, verbose=False, lossfunction = LossRobustBIC)
    summary_df = pd.DataFrame.from_dict({
        'subj_id': [subj_id],
        'driftabsent': [float(fit_model.parameters()['drift']['driftabsent'])],
        'driftpresent': [float(fit_model.parameters()['drift']['driftpresent'])],
        'noise': [float(fit_model.parameters()['noise']['noise'])],
        'bound': [float(fit_model.parameters()['bound']['B'])],
        'IC': [float(fit_model.parameters()['IC']['x0'])],
        'nondecpresent': [float(fit_model.parameters()['overlay']['nondectimepresent'])],
        'nondecabsent': [float(fit_model.parameters()['overlay']['nondectimeabsent'])],
        'BIC': [fit_model.fitresult.value()]
    })
    
    if not os.path.exists("model_fits/model_m7/data_"+data_label):
                       os.makedirs("model_fits/model_m7/data_"+data_label)
                       
    with open("model_fits/model_m7/data_"+data_label+"/subj"+str(subj_id)+".txt", "w") as f:
        f.write(repr(fit_model))
    
    return(summary_df)

In [36]:
loaded = pd.read_csv("data/E2.csv") 
loaded['present'] = loaded['present'].replace(-1,0)
summary_list = []
for name, group in loaded.groupby('subj_id'):
    print(name)
    a= fit_m7(group.reset_index(),'E2')
    summary_list.append(a)
summary_df = pd.concat(summary_list)
summary_df.to_csv("model_fits/model_m7/data_E2/all_subjects.csv")

1


Info: Params [ 0.         -1.94030983  0.69793247  0.50168669  0.6095783   0.57921409] gave 101.59308630397221


2


Info: Params [ 0.         -1.38401023  0.90966948  0.2799799   0.83639137  0.98410181] gave 169.46592794966512


3


...

Info: Params [ 0.03466987 -1.39167683  1.38503741  0.163835    0.91387357  0.95844566] gave 205.93685838462426


297


Info: Params [ 0.37177865 -0.89564915  0.97392639  0.02038952  0.92802343  0.9292813 ] gave 194.8945007553816


298


Info: Params [ 0.67863041 -0.31107945  1.31457587  0.12337219  0.90538316  0.9548102 ] gave 239.98837126890638


In [ ]:
models = [0, fit_m1, fit_m2, fit_m3, fit_m4, fit_m5, fit_m6]

for i_data in list(range(3,5)):
    loaded = pd.read_csv("data/E"+str(i_data)+".csv") 
    loaded['present'] = loaded['present'].replace(-1,0)
    for i_model in list(range(1,7)):
        summary_list = []
        for name, group in loaded.groupby('subj_id'):
            print(name)
            fit_result = models[i_model](group.reset_index(),'E2')
            summary_list.append(fit_result)
        summary_df = pd.concat(summary_list)
        if not os.path.exists("model_fits/model_m"+str(i_model)+"/data_E"+str(i_data)):
                   os.makedirs("model_fits/model_m"+str(i_model)+"/data_E"+str(i_data))
        summary_df.to_csv("model_fits/model_m"+str(i_model)+"/data_E"+str(i_data)+"/all_subjects.csv")

1


Info: Params [ 0.40251789 -1.34956371  1.38440093  0.23084832  0.92918123] gave 210.11483117741562


2


Info: Params [ 1.71068887 -1.85767097  0.56922261  0.07482092  0.43234318] gave 23.593523417861512


3


...

44


Info: Params [1.06024449 0.         1.38749999 2.49378261 0.17927082 0.93920587] gave 190.57960495421236


45


Info: Params [0.86587273 0.         1.05263163 2.49562784 0.11272983 0.99195629] gave 231.273596991056


46


In [16]:
from pyddm import FitResult
from pyddm import Fittable, Fitted
import pyddm.plot


def plotSubj(subj_id, model_fit, exp):
    
    with open("model_fits/model_"+model_fit+"/data_"+exp+"/subj"+str(subj_id)+".txt", "r") as f:
        model = eval(f.read())
        
    data_loaded = pd.read_csv("data/"+exp+".csv") 

    pyddm.plot.model_gui(model=model, sample = Sample.from_pandas_dataframe(data_loaded[data_loaded.subj_id==subj_id], rt_column_name="rt", correct_column_name="correct"))


In [46]:
plotSubj(8,'m3','E2')